In [1]:
import vk
import pandas as pd
import time
from datetime import datetime

In [2]:
# opennig vk session with access token.
session = vk.Session(access_token='3408bdb4e993af86ca9c00578293185575944c4ae31c85d19f8f70e954840551f79d83affd322e6e81667')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

[{'first_name': 'Alexander', 'last_name': 'Ovchinnikov', 'uid': 182927367}]

In [3]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [4]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 317 ms


In [5]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [6]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [8]:
#time.sleep(300)
key_words = ['#глаза', 
             '#глазки', '#зрение', '#лечениезрения', '#детскоезрение', '#лечениеглаз',
             '#офтальмолог','#офтальмология', '#офтальмохирург', '#офтальмохирургия', '#детскоезрение',
             '#детскийофтальмолог', '#офтальмологмосква','#дакриоцистит', '#дакриоциститноворожденных',
             '#дакриоциститмассаж','#дакриоциститлечение', '#дакриоцисториностомия', '#зондированиеносослёзногоканала',
             '#зондированиеноворождённых','#нистагм', '#лечениенистагма','#нистагмоперация', '#нистагмъ',
             '#близорукость', '#миопия', '#лечениеблизорукости', '#близорукостьнеприговор',
             '#близоркостьлечение', '#близорукостьонатакая', '#близорукостьнепорок',
             '#близорукостьудетей','#дальнозоркость', '#гиперметропия', '#лечениедальнозоркости',
             '#дальнозоркостьлечение','#дальнозоркостьудетей', '#дальнозоркости','#косоглазие',
             '#страбизм', '#лечениекосоглазия', '#косоглазиелечение', '#косоглазиеэтосексуально',
             '#косоглазиебишкек', '#косоглазиенепорок', '#косоглазие дети', '#косоглазиеэтомодно',
             '#астигматизм', '#астигматизмлечение', '#астигматизмудетей','#амблиопия', '#амблиопиявысокойстепени',
             '#амблиопиямосква', '#катаракта','#катарактавладивосток', '#катарактачелябинск',
             '#катаракта лечение москва','#катарактаоперация', '#глаукома', '#глаукомалечение', 
             '#глаукомавладивосток','#глаукомачелябинск', '#птоз', '#птозверхнеговека', '#птозвека',
             '#птозвек', '#птозанет','#лазернаякоррекция', '#лазернаякоррекциязрения', '#лазернаякоррекциязрениямосква',
             '#лазернаякоррекцияакция', '#лазернаякоррекциямосква', '#лазернаякоррекциязренияомикрон',
             '#лазернаякоррекцияскидки', '#lasik', '#фрк', '#smile', '#ласик', '#ласикмосква', '#ячмень', '#халязион']
desired_time = 86400*1 # how much back we want to go in seconds
entire_df = pd.DataFrame()
k = 1
for key_word in key_words:
    #print('search for keyword: ', key_word)
    date = time.time() # now
    excess_time = True
    all_df = pd.DataFrame()
    c=1
    while excess_time:
        if k%60 == 0:
            #print('Rest for 10 min')
            time.sleep(600)
            print('Work')
         # print iteration count, extended tells us group vs user
        g=api.newsfeed.search(q=[key_word],count=200, end_time = date-1, start_time=1,extended=1)
        if len(g)>1: # check if g contains data
            all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
            date=g[-1]['date']
        elif len(g) <= 1 and c == 1: # check if there is data
            #print('No posts with that keyword(s)!')
            time.sleep(0.4)
            break
        elif len(g) <= 1 and c != 1: # check if there are posts eft to scrape
            #print('No more posts to scrape!')
            break

        if date<time.time()-desired_time: # check termination condition
            excess_time = False
        #print(c,'iteration')
        c+=1
        k+=1
    all_df['keyword'] = key_word
    #print(len(all_df))
    entire_df = pd.concat([entire_df,all_df])
#print(len(entire_df))

In [16]:
print(len(entire_df))

2114


In [8]:
entire_df.to_csv(r'D:\vk_all_keywords_365_days.csv')

In [9]:
#active_users.post_id.value_counts()

In [10]:
#api.likes.getList(type='comment',item_id='15627.0',owner_id='18659664')

In [11]:
#api.wall.getComments(post_id='15627.0',owner_id='18659664')

In [12]:
all_id = pd.DataFrame()
all_df_new_2=entire_df[entire_df.owner_id>0].reset_index()
for part in range(int(len(all_df_new_2)/9000)+1):
    
    # take all positive ids ( user ids , negative ids are group ids)
    all_df_new=all_df_new_2.loc[(part*9000):(1+part)*9000]

    # get all users
    all_id_2 = pd.DataFrame(api.users.get(user_ids=all_df_new['owner_id'].astype(int), fields = ['occupation']))
    all_id = pd.concat([all_id,all_id_2])

# fill active users status with valid
all_id.fillna('Valid',inplace=True) 
try:    
    # select all deactivated user ids
    deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

    # select all active user posts
    active_users = all_df_new[-all_df_new.owner_id.isin(deactivated_ids)] 
except:
    active_users = all_df_new  

In [13]:
active_users.owner_id.value_counts().head()

349341363.0    122
402190387.0     28
335073041.0     22
502331.0         8
96085636.0       6
Name: owner_id, dtype: int64

In [14]:
api.users.get(user_ids=349341363,fields = ['occupation','education','universities',
                                           'bdate','city','interests','activities','about','site'])

[{'about': 'Достижения:\n\n* первым в России имплантировал искусственный хрусталик Acrysof Single Piece\n* 7 октября 2009 года первым в Южном Федеральном округе выполнил 100% лазерную коррекцию зрения Femto Super LASIK\n* среди хирургов Юга России был выбран американской компанией Alcon и получил первый международный сертификат на право имплантации линзы ReSTOR\n* в 2009 год был награжден председателем краевого офтальмологического общества за лучший доклад на научных конференциях\n* награжден  главным офтальмологом края за самую высокую хирургическую активность среди офтальмохирургов Краснодарского края:\n- в 2009 году выполнил 2006 операций, \n- в 2012 году – 2730 операций\n- в 2013 году выполнил 3445 операций, в том числе провел эксимерлазерную коррекцию Femto Super LASIK своей дочери\n\nНаучная работа:\n\n* 48 научных статей, из них 6 в центральных журналах, 2 — в иностранной литературе\n* 5 патентов на собственные методики диагностики и хирургии катаракты\n* автор книги «Оптимизаци

In [15]:
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date','key_word'])
# resetting the index it make circles of 0-200
active_users.reset_index(inplace = True)
for i in tqdm(range(len(active_users))):    
    new_df.loc[i,'key_word'] = active_users.loc[i,'keyword']
    new_df.loc[i,'user_id'] = active_users.user.loc[i]['uid']
    new_df.loc[i,'sex'] = active_users.user.loc[i]['sex']   
    new_df.loc[i,'Name_Surname'] = active_users.user.loc[i]['first_name']+'_'+active_users.user.loc[i]['last_name']
    new_df.loc[i,'post'] = active_users.text.loc[i]
    new_df.loc[i,'likes'] = active_users.likes.loc[i]['count']
    new_df.loc[i,'comments'] = active_users.comments.loc[i]['count'] 
    new_df.loc[i,'reposts'] = active_users.reposts.loc[i]['count']
    try:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']+'_'+active_users.post_source.loc[i]['platform']
    except:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']
    new_df.loc[i,'date'] = datetime.fromtimestamp(active_users.date.loc[i]).strftime('%Y-%m-%d %H:%M:%S')
new_df['sex'] = new_df['sex'].apply(lambda x: 'Female' if x==1 else 'Male' if x==2  else 'Not Specified')
#new_df.head()

100%|███████████████████████████████████████████████████████████████████████████████| 532/532 [00:03<00:00, 163.31it/s]


In [ ]:
new_df.to_csv(r'D:\vk_all_keywords_365_days_pretty.csv')

In [18]:
short_df =new_df.copy()

In [19]:
short_df.shape

(532, 10)

In [12]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.post.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [13]:
not_long = short_df[short_df['post']!='Too Long'].reset_index().drop('index', axis = 1)

In [30]:
import re
list_of_animals = 'кот|кошка|котёнок|котяра|котяры|котяр|котище|котик|собака|пес|собачка|кобель|псина|сука|птица|пташка|птенцы|птенец|щенок'
def find_animal(x):
    my_list = re.findall(list_of_animals,x)
    if len(my_list) == 0:
        return(True)
    else:
        return(False)

In [31]:
short_df['animal'] = short_df.post.apply(lambda x: True if find_animal(x)==True else False)

In [ ]:
short_df.to_csv(r'D:\vk_all_keywords_365_days_short.csv')

In [33]:
short_df.animal.value_counts()

True     79
False     1
Name: animal, dtype: int64